In [5]:
from keras.applications import ResNet152
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from scipy.stats import spearmanr
from xgboost import XGBRegressor

import progressbar
import h5py
import numpy as np

## Generate Labels

In [40]:
f_train = open("lamem/splits/train_1.txt").readlines()
f_test = open("lamem/splits/test_1.txt").readlines()
f_val = open("lamem/splits/val_1.txt").readlines()

X_train_list = [x[:12] for x in f_train]
Y_train = np.array([float(x[13:21]) for x in f_train])

X_test_list = [x[:12] for x in f_test]
Y_test = np.array([float(x[13:21]) for x in f_test])

X_val_list = [x[:12] for x in f_val]
Y_val = np.array([float(x[13:21]) for x in f_val])

In [46]:
with h5py.File("Y.h5", "w") as h5f:
    for i in range(len(Y_train)):
        h5f.create_dataset(X_train_list[i], data=Y_train[i])
    for i in range(len(Y_test)):
        h5f.create_dataset(X_test_list[i], data=Y_test[i])
    for i in range(len(Y_val)):
        h5f.create_dataset(X_val_list[i], data=Y_val[i])

In [32]:
h5f = h5py.File("test_features.h5")

X_test_valide = [x in set(h5f.keys()) for x in X_test_list]
X_train_list = [x for x, y in zip(X_test_list, X_test_valide) if y]
Y_train = [x for x, y in zip(Y_test, X_test_valide) if y]

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


## Load dataset

In [2]:
# with h5py.File("train_features.h5") as h5f:
#     X_train = np.array([h5f[img][:] for img in X_train_list])
h5f = h5py.File("test_features.h5")
X  = np.array([h5f[img][:] for img in list(h5f.keys())])
h5f2 = h5py.File("Y.h5")
Y  = np.array([h5f2[img].value for img in list(h5f.keys())])

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """
/Users/jojo/opt/anaconda3/lib/python3.7/site-package

In [3]:
X.shape

(7757, 2048)

## Prediction

### XGBoost regressor

In [33]:
from sklearn.metrics import make_scorer
def my_custom_loss_func(y_true, y_pred):
    c, p = spearmanr(y_true, y_pred)
    return c
score = make_scorer(my_custom_loss_func, greater_is_better=True)

In [34]:
ALGORITHMS = {
        'model': XGBRegressor(n_jobs=-1),
        'params': {
            "n_estimators": [100, 150, 200], # 'poly', 'sigmoid'
            "max_depth": [4, 5, 6],
            "eta": [0.01],
        }
    }
CV = 3
model = GridSearchCV(
        ALGORITHMS['model'],
        ALGORITHMS['params'],
        cv=CV,
        scoring=score,
        n_jobs=-1,
        )

In [ ]:
model.fit(X[:7000], Y[:7000])

In [ ]:
pred = model.predict(X[7000:])
c, p = spearmanr(Y[7000:], pred)

print("Spearmanr Result: correlation: {:.5f}, p-value: {}".format(c, p))

### SVM

In [65]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
ALGORITHMS = {
        'model': SVR(gamma='scale'),
        'params': {
            "kernel": ['rbf' ], # 'poly', 'sigmoid'
            "C": [0.1, 1.0],
            "epsilon": [0.001, 1.0],
        }
    }

CV = 3
model = GridSearchCV(
            ALGORITHMS['model'],
            ALGORITHMS['params'],
            cv=CV,
            n_jobs=-1,
        )

In [67]:
model.fit(X[:7000], Y[:7000])

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.1, 1.0], 'epsilon': [0.001, 1.0],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [79]:
from scipy.stats import spearmanr
spearmanr(Y[7000:], pred)

SpearmanrResult(correlation=0.6245817568289069, pvalue=3.988759621179673e-83)

In [31]:
with h5py.File("test_features_new.h5", 'w') as h5f2:
    for i in set(h5f.keys()):
        try:
            h5f2.create_dataset(i, data=h5f[i].value)
        except:
            pass

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  after removing the cwd from sys.path.
